In [6]:
import gym
import numpy as np
import math
import mathplotlib.pyplot as plt

ModuleNotFoundError: No module named 'mathplotlib'

In [ ]:
env = gym.make('CartPole-v0')
env.reset()
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[ 0.04706003  0.02535139 -0.01340691 -0.03182226]
[ 0.04756706  0.22066301 -0.01404335 -0.32870486]
[ 0.05198032  0.02574377 -0.02061745 -0.04048345]
[ 0.0524952  -0.16907655 -0.02142712  0.24562383]
[ 0.04911366  0.02634477 -0.01651464 -0.05373997]
[ 0.04964056  0.22169958 -0.01758944 -0.3515873 ]
[ 0.05407455  0.02683213 -0.02462119 -0.06450237]
[ 0.05461119  0.22229826 -0.02591124 -0.3648506 ]
[ 0.05905716  0.02755396 -0.03320825 -0.08044925]
[ 0.05960824  0.22313583 -0.03481723 -0.38342175]
[ 0.06407095  0.41873437 -0.04248567 -0.68687594]
[ 0.07244565  0.6144195  -0.05622319 -0.99262583]
[ 0.08473403  0.8102468  -0.0760757  -1.3024236 ]
[ 0.10093897  0.61616796 -0.10212418 -1.0344919 ]
[ 0.11326233  0.42254126 -0.12281401 -0.7755375 ]
[ 0.12171315  0.2293033  -0.13832477 -0.5238804 ]
[ 0.12629922  0.42607316 -0.14880237 -0.85675395]
[ 0.13482068  0.23325783 -0.16593745 -0.61431366]
[ 0.13948584  0.04079562 -0.17822373 -0.37814787]
[ 0.14030175 -0.15140685 -0.18578668 -0.14653169]
